# STAC Metadata Visualizations

This notebook illustrates a simple way to display footprints of discovered Items after searching a STAC API, and making simple plots using Pandas and Holoviews. Only the metadata is visualized in these examples through maps and plots. The actual STAC data (i.e., Item Assets) are not accessed.

The libraries GeoPandas and hvplot are used for visualizations.

In [ ]:
from pystac_client import Client

# set pystac_client logger to DEBUG to see API calls
import logging
logging.basicConfig()
logger = logging.getLogger('pystac_client')
logger.setLevel(logging.DEBUG)

Define the STAC API to use, along with any custom headers (such as for authentication)

In [ ]:
# STAC API root URL
URL = 'https://planetarycomputer.microsoft.com/api/stac/v1'

# custom headers
headers = []

cat = Client.open(URL, headers=headers)
cat

## Search

Perform a spatio-temporal search of ASTER data for a small AOI in the northern part of The Netherlands between 2000 and 2010.

In [ ]:
# AOI around Delfzijl, in northern Netherlands
geom = {
    "type": "Polygon",
    "coordinates": [
      [
        [
          6.42425537109375,
          53.174765470134616
        ],
        [
          7.344360351562499,
          53.174765470134616
        ],
        [
          7.344360351562499,
          53.67393435835391
        ],
        [
          6.42425537109375,
          53.67393435835391
        ],
        [
          6.42425537109375,
          53.174765470134616
        ]
      ]
    ]
}

# limit sets the # of items per page so we can see multiple pages getting fetched
search = cat.search(
    max_items = 50,
    collections = "aster-l1t",
    intersects = geom,
    datetime = "2000-01-01/2010-12-31",
)

# retrieve the items as dictionaries, rather than Item objects
items = list(search.items_as_dicts())
len(items)

## GeoPandas

A GeoDataFrame is constructed from the AOI geometry, in order to make visualizations easy.

The STAC Items, which are a GeoJSON FeatureCollection can be converted to a GeoDataFrame.

In [ ]:
from copy import deepcopy
import geopandas as gpd
import pandas as pd
from shapely.geometry import shape

# convert a list of STAC Items into a GeoDataFrame
def items_to_geodataframe(items):
    _items = []
    for i in items:
        _i = deepcopy(i)
        _i['geometry'] = shape(_i['geometry'])
        _items.append(_i)
    gdf = gpd.GeoDataFrame(pd.json_normalize(_items))
    for field in ['properties.datetime', 'properties.created', 'properties.updated']:
        if field in gdf:
            gdf[field] = pd.to_datetime(gdf[field])
    gdf.set_index('properties.datetime', inplace=True)
    return gdf

In [ ]:
# convert geometry to a GeoDataFrame
aoi_gdf = gpd.GeoDataFrame([{'geometry': shape(geom)}])
aoi_gdf

In [ ]:
# convert found items to a GeoDataFrame
items_gdf = items_to_geodataframe(items)
items_gdf.head()

## Plot Geometries on a Map

Holoviews is used to display geometries on a map by using `hvplot`. The The `*` Holoviews operator to overlay two plots

In [ ]:
import hvplot.pandas

# plot polygons as lines on a map with background tiles.
def plot_polygons(data, *args, **kwargs):
    return data.hvplot.paths(*args, geo=True, tiles='OSM', xaxis=None, yaxis=None,
                             frame_width=600, frame_height=600,
                             line_width=4, **kwargs)

In [ ]:
plot_polygons(aoi_gdf)

In [ ]:
plot_polygons(items_gdf)

In [ ]:
plot_polygons(aoi_gdf) * items_gdf.hvplot.paths(geo=True)

## Line Plots

Numeric STAC metadata can also be plotted, most often this will be plotted vs the Item `datetime`.

In [ ]:
items_df = pd.DataFrame(items_gdf)

plot_fields = [
    'properties.eo:cloud_cover',
    'properties.view:sun_azimuth',
    'properties.view:sun_elevation'
]

plots = None
for f in plot_fields:
    if f in items_df:
        if plots is None:
            plots = items_df.hvplot(y=f, label=f.replace('properties.', ''),
                                    frame_height=500, frame_width=800)
        else:
            plots = plots * items_df.hvplot(y=f, label=f.replace('properties.', ''))

plots.opts(ylabel='')